In [19]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
import sys
sys.path.append('/content/gdrive/MyDrive/collab/NLP/argmining-21-keypoint-analysis-sharedtask-code-master') 
os.environ['CUDA_VISIBLE_DEVICES'] = '4'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [20]:
!pip install rouge
!pip install fast_pagerank
!pip install sentence_transformers
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [23]:
!nvidia-smi

Tue May 24 23:05:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [24]:
!pip3 install torch torchvision torchaudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
import torch
torch.cuda.is_available()

False

In [26]:
print(torch.version.cuda)

11.3


In [27]:
import rouge

In [28]:
evaluator = rouge.Rouge(metrics=['rouge-l'])#,
                           #max_n=4,
                           #limit_length=True,
                           #length_limit=100,
                           #length_limit_type='words',
                           #alpha=0.5, # Default F1_score
                           #weight_factor=1.2,
                           #stemming=True)

In [29]:
import json
import pandas as pd
import numpy as np
import spacy
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity


from scipy import sparse
from fast_pagerank import pagerank
from fast_pagerank import pagerank_power


pd.set_option('display.max_colwidth', None)

In [30]:
from sentence_transformers import SentenceTransformer, InputExample, LoggingHandler, losses, models, util
import torch

In [31]:
train_args = pd.read_pickle('/content/gdrive/MyDrive/collab/NLP/argmining-21-keypoint-analysis-sharedtask-code-master/data/train_args_with_scores.pkl')

In [32]:
train_topic_with_scored_kps = pd.read_pickle('/content/gdrive/MyDrive/collab/NLP/argmining-21-keypoint-analysis-sharedtask-code-master/data/train_topic_keypoints_with_scores.pkl')

In [33]:
train_topics = train_args.groupby(['topic', 'stance']).agg({
    'sents_with_scores': lambda x: set([item for items in x for item in items]),
    'gt-kps' : lambda x: set([item for items in x for item in items]),
    #'gt-cand-kps': lambda x: set([item for items in x for item in items])
}).reset_index()

In [34]:
train_topics.columns

Index(['topic', 'stance', 'sents_with_scores', 'gt-kps'], dtype='object')

In [35]:
train_topics.head()

topic  stance  \
0  Assisted suicide should be a criminal offence      -1   
1  Assisted suicide should be a criminal offence       1   
2                 Homeschooling should be banned      -1   
3                 Homeschooling should be banned       1   
4        The vow of celibacy should be abandoned      -1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [ ]:
#TODO: Check if the gt-kps appears inside sents_with_scores correctly, it seems like something wrong there..

In [36]:
#Add to the candidate sentences from the topic, the topic key-points
train_topics['sents_with_scores'] = train_topics.apply(lambda row: list(row['sents_with_scores']) + train_topic_with_scored_kps[(train_topic_with_scored_kps.topic == row['topic']) & (train_topic_with_scored_kps.stance == row['stance'])]['scored_kps'].tolist()[0], axis=1)

In [37]:
model = SentenceTransformer('/content/gdrive/MyDrive/collab/NLP/argmining-21-keypoint-analysis-sharedtask-code-master/siamese-nn/modelroberta-base-contrastive-10-epochs-2022-05-24_11-25-10')

In [42]:
import itertools

def gen_match_matrix(model, sents, topic, min_match_score=0):
    sents1 = [topic + ' <SEP> ' + x for x in sents]
    sents1_embeddings = model.encode(sents1)
    sents_embeddings  = model.encode(sents)
    
    sim_matrix = cosine_similarity(sents1_embeddings, sents1_embeddings)
    super_threshold_indices = sim_matrix < min_match_score
    sim_matrix[super_threshold_indices] = 0

    return sim_matrix

def evalute_topic_by_rouge(row):
    ranked_sents = [x[0] for x in row['ranked_sents']][0:10]
    gt_kps = [[x] for x in row['gt-kps']]

    r_l = []
    for a,b in itertools.product(gt_kps, ranked_sents):
        scores = evaluator.get_scores(a, b)
        r_l.append(scores['rouge-l']['f'])
    
    return np.mean(r_l)

def evalute_topic(row):
    ranked_sents = [x[0] for x in row['ranked_sents']]
    #ranked_sents = [x for x in row['filtered_sents']]
    gt_kps_ranks = []
    for real_kp in row['gt-kps']:
        if real_kp in ranked_sents:
            gt_kps_ranks.append(ranked_sents.index(real_kp)+1)
        else:
            print('kp is not there...')
            gt_kps_ranks.append(len(ranked_sents)+1)
            
    #print(gt_kps_ranks)
    return 1/(sum(gt_kps_ranks)/len(gt_kps_ranks)) if len(gt_kps_ranks) > 0 else 0

In [43]:
def apply_page_rank(row, p=0.85, min_quality_score=0.5, min_match_score=0.5, min_len=5, max_len=20):
    cand_sents = [x for x in row['sents_with_scores'] if x[1] > min_quality_score and len(x[0].split()) < max_len and len(x[0].split()) > min_len]
    if len(cand_sents) == 0:
        return []
        print('empty')
    cands, cands_qualities = zip(*cand_sents)
    #print(cands)
    #print(cands_qualities)
    cands_matching_mat = gen_match_matrix(model, cands, row['topic'], min_match_score=min_match_score)
    pr=pagerank(cands_matching_mat, personalize=np.array(cands_qualities), p=p) #it looks like modifying the initial probability doesn't help
    #pr=pagerank(cands_matching_mat, p=p)
    ranked_candidates = list(zip(cands, pr))
    return sorted(ranked_candidates, key=lambda x: -x[1])    

In [44]:
def filter_ranked_list(row, model, min_match=0.8):
    ranked_sents = [x[0] for x in row['ranked_sents']]
    filtered_sents = []
    while len(filtered_sents) < 5:
        filtered_sents = []
        for i, s in enumerate(ranked_sents):
            if len(filtered_sents) == 0:
                filtered_sents.append(s)
                print('FFFF ' , s)
            else:
                matching_scores = gen_match_matrix(model, [s]+filtered_sents, row['topic'])
                max_sim = np.max(matching_scores[0][1:])
                #print(matching_scores)
                if max_sim < min_match:
                    print('SSSS ' , s)
                    filtered_sents.append(s)
        
        min_match = min_match + 0.1
        print(min_match)

    return filtered_sents

In [ ]:
scores = []
for min_match_score in [0.2, 0.4, 0.6, 0.8, 1.0]:
    for p in [0.2, 0.4, 0.8, 1.0]:
        for min_quality_score in [0.2, 0.5, 0.8]:
            train_topics['ranked_sents'] = train_topics.apply(lambda row: apply_page_rank(row, p=p, min_quality_score=min_quality_score, min_match_score=min_match_score), axis=1)
            train_topics['scores'] = train_topics.apply(lambda row: evalute_topic_by_rouge(row), axis=1)
            score = np.mean(train_topics.scores.tolist())
            scores.append([p, min_quality_score, min_match_score, score])

In [ ]:
scores_df = pd.DataFrame(scores, columns=['p', 'min-quality-score', 'min-match-score', 'score'])
scores_df.sort_values('score', ascending=False)

,p,min-quality-score,min-match-score,score
14,0.2,0.8,0.4,0.243984
12,0.2,0.2,0.4,0.243273
13,0.2,0.5,0.4,0.243273
4,0.4,0.5,0.2,0.243232
3,0.4,0.2,0.2,0.243144
17,0.4,0.8,0.4,0.243108
38,0.2,0.8,0.8,0.242880
5,0.4,0.8,0.2,0.242583
15,0.4,0.2,0.4,0.242096
16,0.4,0.5,0.4,0.242078


In [ ]:
#train_topics['filtered_sents'] = train_topics.apply(lambda row: filter_ranked_list(row, model, min_match=0.3), axis=1)

-------------

##### Apply on test:

In [ ]:
test_kps = pd.read_csv('../../KPA_2021_shared_task/test_data/key_points_test.csv')

In [ ]:
test_args = pd.read_pickle('/workspace/ceph_data/keypoint-analysis-sharedtask/siamese-data/data-with-scores/test_args_with_scores.pkl')

In [ ]:
test_args.head()

,arg_id,argument,topic,stance,sents,sents_with_scores
0,arg_0_0,Routine child vaccinations isn't mandatory since children don't spread the virus,Routine child vaccinations should be mandatory,-1,[Routine child vaccinations isn't mandatory since children don't spread the virus],"[(Routine child vaccinations isn't mandatory since children don't spread the virus, 0.7141945362091064)]"
1,arg_0_1,Routine child vaccinations should not be mandatory because children may not bear the side effects of it.,Routine child vaccinations should be mandatory,-1,[Routine child vaccinations should not be mandatory because children may not bear the side effects of it.],"[(Routine child vaccinations should not be mandatory because children may not bear the side effects of it., 0.871067464351654)]"
2,arg_0_2,Routine child vaccinations should not be necessary as children can't catch the disease,Routine child vaccinations should be mandatory,-1,[Routine child vaccinations should not be necessary as children can't catch the disease],"[(Routine child vaccinations should not be necessary as children can't catch the disease, 0.8207754492759705)]"
3,arg_0_3,A vaccine that has not been sufficiently tested and without knowledge of side effects is not recommended for children,Routine child vaccinations should be mandatory,-1,[A vaccine that has not been sufficiently tested and without knowledge of side effects is not recommended for children],"[(A vaccine that has not been sufficiently tested and without knowledge of side effects is not recommended for children, 0.8951572775840759)]"
4,arg_0_4,"As long as vaccines are not free of side effects, it cannot make them mandatory for our children.",Routine child vaccinations should be mandatory,-1,"[As long as vaccines are not free of side effects, it cannot make them mandatory for our children.]","[(As long as vaccines are not free of side effects, it cannot make them mandatory for our children., 0.8620935082435608)]"


In [ ]:
test_topics = test_args.groupby(['topic', 'stance']).agg({
    'sents_with_scores': lambda x: set([item for items in x for item in items])
}).reset_index()

In [ ]:
test_topics['ranked_sents'] = test_topics.apply(lambda row: apply_page_rank(row, p=0.2, min_quality_score=0.8, min_match_score=0.8), axis=1)

/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseE

In [ ]:
test_topics['ranked_kps'] = test_topics.apply(lambda row: filter_ranked_list(row, model, min_match=0.8), axis=1)

FFFF  vaccination should exclude children to avoid the side effects that can appear on them.
SSSS  parents should have the freedom to decide what they consider best for their children
SSSS  The child population has a low degree of vulnerability, so vaccination is not urgent yet
SSSS  When vaccines are mandatory, they can infringe on family religious choices
SSSS  Linking a measure as good as vaccination to coercive measures would cause serious harm
SSSS  Making child vaccination compulsory is not the the best way to obtain high vaccination rates
SSSS  vaccination of children should be limited as it affects child development
SSSS  child vaccinations is not mandatory because it may cost the country unnecessary funds.
SSSS  the poorest people may not be able to pay for the vaccine
SSSS  Forcing people to have their children vaccinated goes against free will.
SSSS  Vaccines contain heavy metals, they are toxic to children.
0.9
FFFF  child vaccinations should be mandatory to provide decent 

In [ ]:
#adding gt-kps
test_topics['gt-kps']= test_topics.apply(lambda row: test_kps[(test_kps.topic==row['topic']) & (test_kps.stance==row['stance'])]['key_point'].tolist(), axis=1)

In [ ]:
test_topics.head()

topic  stance  \
0                Routine child vaccinations should be mandatory      -1   
1                Routine child vaccinations should be mandatory       1   
2  Social media platforms should be regulated by the government      -1   
3  Social media platforms should be regulated by the government       1   
4                          The USA is a good country to live in      -1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [ ]:
test_topics[['topic', 'stance', 'ranked_kps', 'gt-kps']].to_pickle('../../data/pagerank-test-keypoints.pkl')

In [ ]:
keypoints_ds = []
c = 0
for i, row in test_topics.iterrows():
    for kp in row['ranked_kps'][0:5]:
        keypoints_ds.append([c, row['topic'], row['stance'], kp])
        c+=1
        
keypoints_df = pd.DataFrame(keypoints_ds, columns=['key_point_id', 'topic', 'stance', 'key_point'])

In [ ]:
keypoints_df.sample(n=5).head()

,key_point_id,topic,stance,key_point
23,23,The USA is a good country to live in,-1,The United States has long been a hotbed of racism against minority cultures
2,2,Routine child vaccinations should be mandatory,-1,"The child population has a low degree of vulnerability, so vaccination is not urgent yet"
27,27,The USA is a good country to live in,1,The USA economy is one of the best in the world especially for job opportunities
22,22,The USA is a good country to live in,-1,The USA is a very capitalist country; it does not provide universal health care or education for its citizens.
29,29,The USA is a good country to live in,1,The USA has a well established and stable political system


In [ ]:
ls ../../

KPA_2021_shared_task/  notes.txt      src-ipynb/
README.md              paraphrasing/  src-py/
data/                  siamese-nn/    track-2-keypoint-extraction/


In [ ]:
keypoints_df.to_csv('../../track-2-keypoint-extraction/pagerank-generated-test-keypoints.csv', index=False)

In [ ]:
keypoints_df.topic.value_counts()

Social media platforms should be regulated by the government    10
Routine child vaccinations should be mandatory                  10
The USA is a good country to live in                            10
Name: topic, dtype: int64